In [201]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
import lib as lib

### Solution 3:
We use random forest algorithm to predict max user and bandwidth independently<br>
The first model is used to predict maxUser  base on data: **"ZONE_CODE", "HOUR_ID", "DATE_IN_WEEK", "DAY_SHIFT"** <br>
The second model is used to predict bandwidth base on data: **MAX_USER** <br>
Comebine these two models to predict maxUser and bandwidth on **test_id.csv**

In [202]:
dataset = pd.read_csv("data/clean_dataset.csv")

In [203]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35544 entries, 0 to 35543
Data columns (total 9 columns):
Unnamed: 0         35544 non-null int64
UPDATE_TIME        35544 non-null int64
ZONE_CODE          35544 non-null int64
HOUR_ID            35544 non-null int64
BANDWIDTH_TOTAL    35544 non-null float64
MAX_USER           35544 non-null float64
DATE_IN_MONTH      35544 non-null int64
DATE_IN_WEEK       35544 non-null int64
DAY_SHIFT          35544 non-null int64
dtypes: float64(2), int64(7)
memory usage: 2.4 MB


In [204]:
dataset.head(10)

,Unnamed: 0,UPDATE_TIME,ZONE_CODE,HOUR_ID,BANDWIDTH_TOTAL,MAX_USER,DATE_IN_MONTH,DATE_IN_WEEK,DAY_SHIFT
0,0,1506816000,1,0,16096.710313,212415.0,1,6,0
1,1,1506816000,1,1,9374.207907,166362.0,1,6,0
2,2,1506816000,1,2,5606.225750,146370.0,1,6,0
3,3,1506816000,1,3,4155.654661,141270.0,1,6,0
4,4,1506816000,1,4,3253.978594,139689.0,1,6,0
5,5,1506816000,1,5,4310.774522,165342.0,1,6,0
6,6,1506816000,1,6,10015.599874,221901.0,1,6,0
7,7,1506816000,1,7,19780.995631,302736.0,1,6,0
8,8,1506816000,1,8,24344.875025,336600.0,1,6,1
9,9,1506816000,1,9,28651.427689,356235.0,1,6,1


In [205]:
dataset.corr()

,Unnamed: 0,UPDATE_TIME,ZONE_CODE,HOUR_ID,BANDWIDTH_TOTAL,MAX_USER,DATE_IN_MONTH,DATE_IN_WEEK,DAY_SHIFT
Unnamed: 0,1.000000,0.999740,0.032327,0.002138,0.233774,0.154339,-0.007666,0.004118,0.001711
UPDATE_TIME,0.999740,1.000000,0.032960,0.001460,0.233044,0.154116,-0.006745,0.004290,0.001073
ZONE_CODE,0.032327,0.032960,1.000000,0.000041,-0.611186,-0.755793,0.008337,-0.001165,0.000036
HOUR_ID,0.002138,0.001460,0.000041,1.000000,0.313592,0.181726,-0.000790,0.000046,0.943551
BANDWIDTH_TOTAL,0.233774,0.233044,-0.611186,0.313592,1.000000,0.925587,0.003470,0.035023,0.309379
MAX_USER,0.154339,0.154116,-0.755793,0.181726,0.925587,1.000000,0.008498,0.002566,0.183248
DATE_IN_MONTH,-0.007666,-0.006745,0.008337,-0.000790,0.003470,0.008498,1.000000,-0.017459,-0.000486
DATE_IN_WEEK,0.004118,0.004290,-0.001165,0.000046,0.035023,0.002566,-0.017459,1.000000,-0.000120
DAY_SHIFT,0.001711,0.001073,0.000036,0.943551,0.309379,0.183248,-0.000486,-0.000120,1.000000


In [206]:
#data = dataset[['DATE_IN_MONTH', 'ZONE_CODE', 'HOUR_ID']]
data = dataset[['ZONE_CODE', 'HOUR_ID', 'DATE_IN_WEEK', "DAY_SHIFT"]]
bandwidth_total = dataset.BANDWIDTH_TOTAL
max_user = dataset.MAX_USER

In [207]:
#x_train, x_test, y_train, y_test = train_test_split(data, max_user,test_size=0.3)
#x_train_max_user, x_test_max_user, y_train_max_user, y_test_max_user = train_test_split(data, max_user,test_size=0.3)

In [208]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [209]:
model_maxUser = RandomForestRegressor(n_estimators = 100, max_features='auto')
model_maxUser.fit(data, max_user)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [210]:
#predict_max_user = model_maxUser.predict(data)

In [211]:
x_train_maxuser, x_test_maxuser, y_train_bandwidth, y_test_bandwidth = train_test_split(pd.DataFrame(max_user), bandwidth_total, test_size=0.3)

In [212]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
list_k = []
list_acc = []
for k in range(2,200):
    list_k.append(k)
    neigh = KNeighborsRegressor(n_neighbors = k, weights='uniform', algorithm='auto')
    neigh.fit(x_train_maxuser,  y_train_bandwidth)
    y_pred_banwidth = neigh.predict(x_test_maxuser)
    acc = r2_score(y_test_bandwidth, y_pred_banwidth)
    list_acc.append(acc)
    print("The Accurcy is", acc * 100, "% for k-value", k)
index = list_acc.index(max(list_acc))
k = list_k[index]
print("\nThe optimal number of neighbors is %d with %f%%" % (k, list_acc[index]* 100))


The Accurcy is 83.96852551819201 % for k-value 2
The Accurcy is 85.91673300264485 % for k-value 3
The Accurcy is 86.97857369057168 % for k-value 4
The Accurcy is 87.43931967521615 % for k-value 5
The Accurcy is 87.71227439127772 % for k-value 6
The Accurcy is 87.96976317234042 % for k-value 7
The Accurcy is 88.14290226683438 % for k-value 8
The Accurcy is 88.27451957989638 % for k-value 9
The Accurcy is 88.36216833866976 % for k-value 10
The Accurcy is 88.38851471500556 % for k-value 11
The Accurcy is 88.4615881590755 % for k-value 12
The Accurcy is 88.55686107456108 % for k-value 13
The Accurcy is 88.60701341947906 % for k-value 14
The Accurcy is 88.69357104437118 % for k-value 15
The Accurcy is 88.75735097477373 % for k-value 16
The Accurcy is 88.79443496685957 % for k-value 17
The Accurcy is 88.81438031422275 % for k-value 18
The Accurcy is 88.88697696248282 % for k-value 19
The Accurcy is 88.88942883687498 % for k-value 20
The Accurcy is 88.8554579800694 % for k-value 21
The Accurc

The Accurcy is 89.06060710494872 % for k-value 166
The Accurcy is 89.05712206966021 % for k-value 167
The Accurcy is 89.06063560858225 % for k-value 168
The Accurcy is 89.0613664208746 % for k-value 169
The Accurcy is 89.05954499351022 % for k-value 170
The Accurcy is 89.0585142961723 % for k-value 171
The Accurcy is 89.0633000498551 % for k-value 172
The Accurcy is 89.057663332373 % for k-value 173
The Accurcy is 89.05909459179637 % for k-value 174
The Accurcy is 89.05928038868043 % for k-value 175
The Accurcy is 89.05552666570313 % for k-value 176
The Accurcy is 89.05351417253424 % for k-value 177
The Accurcy is 89.05333771287836 % for k-value 178
The Accurcy is 89.0501099685713 % for k-value 179
The Accurcy is 89.04664144295168 % for k-value 180
The Accurcy is 89.05242534638529 % for k-value 181
The Accurcy is 89.04996161761804 % for k-value 182
The Accurcy is 89.04617573500141 % for k-value 183
The Accurcy is 89.04436631457021 % for k-value 184
The Accurcy is 89.04355007732664 % fo

In [213]:
model_bandwidth = KNeighborsRegressor(n_neighbors = k, weights='uniform', algorithm='auto')
model_bandwidth.fit(pd.DataFrame(max_user),  bandwidth_total)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=95, p=2,
                    weights='uniform')

In [214]:
raw_test_data = pd.read_csv("data/test_id.csv")

In [215]:
raw_test_data.head()

,id,UPDATE_TIME,ZONE_CODE,HOUR_ID
0,0,2019-03-10,ZONE01,0
1,1,2019-03-10,ZONE01,1
2,2,2019-03-10,ZONE01,2
3,3,2019-03-10,ZONE01,3
4,4,2019-03-10,ZONE01,4


In [216]:
raw_test_data.ZONE_CODE = raw_test_data.ZONE_CODE.map({'ZONE01' : 1, 'ZONE03' : 3, 'ZONE02' : 2})
raw_test_data['DATE_IN_MONTH'] =  raw_test_data.UPDATE_TIME.map(lib.get_date_in_month).astype('int32')
raw_test_data['DATE_IN_WEEK'] =  raw_test_data.UPDATE_TIME.map(lib.get_date_in_week).astype('int32')
raw_test_data['DAY_SHIFT'] =  raw_test_data.HOUR_ID.map(lib.hour_to_shift).astype('int32')
raw_test_data.UPDATE_TIME = raw_test_data.UPDATE_TIME.map(lib.convert_datetime_to_timestamp).astype('int32')

In [217]:
raw_test_data.head()

,id,UPDATE_TIME,ZONE_CODE,HOUR_ID,DATE_IN_MONTH,DATE_IN_WEEK,DAY_SHIFT
0,0,1552176000,1,0,10,6,0
1,1,1552176000,1,1,10,6,0
2,2,1552176000,1,2,10,6,0
3,3,1552176000,1,3,10,6,0
4,4,1552176000,1,4,10,6,0


In [218]:
test_data = raw_test_data[['ZONE_CODE', 'HOUR_ID', 'DATE_IN_WEEK', "DAY_SHIFT"]]

In [219]:
test_maxUser = model_maxUser.predict(test_data)

In [220]:
test_bandwidth = model_bandwidth.predict(pd.DataFrame(test_maxUser))

In [221]:
test_bandwidth = np.around(test_bandwidth, decimals=2)
test_bandwidth

array([29842.19, 17604.3 , 12675.14, ...,  3678.35,  3752.57,  1590.13])

In [222]:
test_maxUser

array([440019.28068655, 330785.66658986, 279625.67573317, ...,
        49555.88349485,  40443.50964735,  30990.18810322])

In [223]:
test_maxUser = np.round(test_maxUser)
test_maxUser

array([440019., 330786., 279626., ...,  49556.,  40444.,  30990.])

In [224]:
final_test_data = pd.DataFrame({'id': raw_test_data['id'], 'bandwidth': test_bandwidth,"max_user": test_maxUser})

In [225]:
final_test_data['label'] = final_test_data[['bandwidth', 'max_user']].apply(lambda x: '{} {}'.format(x[0],x[1]), axis=1)

In [226]:
export_test_data = final_test_data[['id', 'label']]

In [227]:
export_test_data.to_csv("data/solution_random_forest_relative_prediction.csv", index=False)

In [228]:
export_test_data.head()

,id,label
0,0,29842.19 440019.0
1,1,17604.3 330786.0
2,2,12675.14 279626.0
3,3,10656.44 249368.0
4,4,10832.51 246221.0
